In [1]:
%load_ext autoreload
%autoreload 2

import re
from glob import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
import matplotlib
plt.style.use('ggplot')
matplotlib.rcParams['figure.figsize'] = 12, 6
matplotlib.rcParams['font.size'] = 18  # Probably OS Dependent

In [3]:
data_files = glob('../public/*.json')

def valid_element(el_list):
    values = [len(x.strip()) > 0 for x in el_list if x]
    if len(values) == 0:
        return False
    return any(values)
    

data = []
for file_ in data_files:
    df = pd.read_json(file_)
    false_series = pd.Series(False, index=df.index)
    valid_email = df['emails'].apply(valid_element) if 'emails' in df else false_series
    valid_fax = df['faxes'].apply(valid_element) if 'faxes' in df else false_series
    valid_email | valid_fax

    data += [{
        'state': re.search('public/(.+).json', file_).group(1).title(),
        'records': df.shape[0],
        'fields': df.shape[1],
        'valid_locale': df['locale'].apply(valid_element).mean(),
        'valid_city': df['city'].apply(valid_element).mean() if 'city' in df else 0.,
        'valid_county': df['county'].apply(valid_element).mean() if 'county' in df else 0.,
        'valid_official': df['official'].apply(valid_element).mean(),
        'valid_official': df['official'].apply(valid_element).mean(),
        'valid_email': valid_email.mean(),
        'valid_faxes': valid_fax.mean(),
        'valid_fax_or_email': (valid_email | valid_fax).mean(),
    }]

pd.DataFrame(data)


,state,records,fields,valid_locale,valid_city,valid_county,valid_official,valid_email,valid_faxes,valid_fax_or_email
0,Nebraska,93,8,1.0,0.0,1.0,1.000000,1.000000,1.000000,1.000000
1,Georgia,159,6,1.0,0.0,1.0,1.000000,0.968553,0.987421,0.993711
2,Wisconsin,1853,9,1.0,1.0,1.0,1.000000,0.861846,0.546681,0.950890
3,Maine,504,6,1.0,1.0,0.0,1.000000,0.000000,0.942460,0.942460
4,Virginia,133,7,1.0,0.0,0.0,1.000000,1.000000,1.000000,1.000000
5,Maryland,24,8,1.0,0.0,1.0,0.916667,1.000000,0.000000,1.000000
6,Michigan,1535,7,1.0,1.0,1.0,1.000000,0.981107,0.687296,0.988925
7,Florida,67,5,1.0,0.0,1.0,1.000000,1.000000,0.000000,1.000000
8,Minnesota,87,6,1.0,0.0,1.0,1.000000,1.000000,1.000000,1.000000
